In [3]:
import os
from dotenv import dotenv_values
from cassiopeia import Patch
from SQDBI.database import Session

# Load environment variables
config = {
    **dotenv_values(".sheet.env"),
    **os.environ,
}

# config.update(dotenv_values(".env.admin"))

os.environ.update(config)

import gspread
gc = gspread.service_account(filename=os.environ.get("SHEET_CLIENT_SECRET"))
sh = gc.open_by_key(os.environ.get("TEAM_SHEET_ID"))

DEFAULT_REGION = sh.worksheet('validators').acell('E2').value
# PLAYER_ACCOUNTS = update_player_accounts(sh, RIOT_API)

MATCH_WKS = sh.worksheet('matches')
# IMPORTED_MATCH_IDS = MATCH_WKS.get_as_df()[["player", "match"]].groupby('player')['match'].apply(lambda x: set(x.unique())).to_dict()

SEASON_START = Patch.from_str("14.1", region=DEFAULT_REGION).start

# Update latest 3 patches
latest_patch = Patch.latest(DEFAULT_REGION)
patch = [f"{latest_patch.major}.{int(latest_patch.minor) - i}" for i in range(3)]
sh.worksheet('validators').update('C2', list(map(list, zip(patch))))
# We now need to retrieve the matches that haven't been updated in the google sheet (we can assume that our database is always updated)

Making call: https://cdn.merakianalytics.com/riot/lol/resources/patches.json


AttributeError: 'Worksheet' object has no attribute 'update_values'

In [ ]:
from sqlalchemy import select, tuple_, and_ 
from SQDBI.models import Participant, Player, Game, Account


from sqlalchemy import select, tuple_

with Session() as session:
    statement = (
        select(
            Participant.account_puuid,
            Game.game_version_major,
            Game.game_version_minor,
            Participant.game_id,
            Game.game_start,
            Participant.champion_id,
            Participant.lane,
            Participant.kda,
            Participant.puuid,
            Participant.win
        )
        .join(Game, Participant.game)
        .outerjoin(Account, Participant.puuid)
        .where(Participant.account_puuid is not None)
        # .where(~tuple_(Participant.account_name, Participant.account_tagline).in_(imported_matches))
        .limit(10)
    )
    
    results = session.execute(statement).all()
    for row in results:
        print(row)

AttributeError: mapper

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\icosi\AppData\Local\Programs\Python\Python312\Lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\icosi\AppData\Local\Programs\Python\Python312\Lib\asyncio\selector_events.py", line 132, in _read_from_self
    data = self._ssock.recv(4096)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\icosi\AppData\Local\Programs\Python\Python312\Lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\icosi\AppData\Local\Programs\Python\Python312\Lib\asyncio\selector_even

In [6]:
from sqlalchemy import select, tuple_, and_ 
from SQDBI.models import Participant, Player, Game

from sqlalchemy import select, tuple_

with Session() as session:
    statement = (
        select(
            Game.game_version,
            Game.game_version_major,
            Game.game_version_minor,
        )
        .limit(100)
    )
    # print(statement)
    results = session.execute(statement).all()
    for row in results:
        print(row)

('14.6.570.6276', 14, 6)
('14.6.570.6276', 14, 6)
('14.6.570.6276', 14, 6)
('14.6.570.6276', 14, 6)
('14.6.570.6276', 14, 6)
('14.6.570.6276', 14, 6)
('14.6.570.6276', 14, 6)
('14.6.570.6276', 14, 6)
('14.6.570.6276', 14, 6)
('14.6.570.6276', 14, 6)
('14.6.570.6276', 14, 6)
('14.6.570.6276', 14, 6)
('14.6.570.6276', 14, 6)
('14.6.570.6276', 14, 6)
('14.6.570.6276', 14, 6)
('14.6.570.6276', 14, 6)
('14.6.570.6276', 14, 6)
('14.6.570.6276', 14, 6)
('14.6.570.6276', 14, 6)
('14.6.570.6276', 14, 6)
('14.6.570.6276', 14, 6)
('14.6.570.6276', 14, 6)
('14.6.569.5139', 14, 6)
('14.6.569.5139', 14, 6)
('14.6.569.5139', 14, 6)
('14.6.569.5139', 14, 6)
('14.6.569.5139', 14, 6)
('14.6.569.5139', 14, 6)
('14.6.569.5139', 14, 6)
('14.6.569.5139', 14, 6)
('14.6.569.5139', 14, 6)
('14.6.569.5139', 14, 6)
('14.6.569.5139', 14, 6)
('14.6.569.5139', 14, 6)
('14.6.569.5139', 14, 6)
('14.6.569.5139', 14, 6)
('14.6.569.5139', 14, 6)
('14.6.569.5139', 14, 6)
('14.6.569.5139', 14, 6)
('14.6.569.5139', 14, 6)


In [2]:
from sqlalchemy import select, update, func, Integer
from SQDBI.database import Session
from SQDBI.models import Game

def update_game_versions():
    with Session() as session:
        # Subquery to extract major and minor versions
        subquery = (
            select(
                Game.id,
                func.cast(func.split_part(Game.game_version, '.', 1), Integer).label('major'),
                func.cast(func.split_part(Game.game_version, '.', 2), Integer).label('minor')
            ).subquery()
        )

        # Update statement
        stmt = (
            update(Game)
            .values(
                game_version_major=subquery.c.major,
                game_version_minor=subquery.c.minor
            )
            .where(Game.id == subquery.c.id)
        )

        # Execute the update
        result = session.execute(stmt)
        session.commit()

        print(f"Updated {result.rowcount} rows.")

update_game_versions()

Updated 14376 rows.


In [51]:
len(results)

NameError: name 'results' is not defined

In [31]:
with Session() as session:
    statement = select(Participant.account_name, Participant.account_tagline).limit(10)
    print(session.execute(statement).all())

[('41205', 'NA1'), ('nothing', '星河滚烫'), ('Mr Master 2020', 'Best1'), ('mlnzgmr', 'tudi'), ('Yüksekçi', 'NA1'), ('', 'NA1'), ('Toothbooth', 'NA1'), ('Mixtoour', 'NA1'), ('big dacko', 'NA1'), ('Pockus', 'NA1')]
